# Deep Learning Project

The current project relies on using CNNs in order to process sperm data. The goal is to develop a classifier able to identify sperm cells.

The model is not yet fuly defined. We will start with a standard CNN to Dense Layer.

The goal is for images to be loaded into the CNN. The CNN will will then perform feature extraction and those will be fed to the final model.

## Step 1 - Create the pipeline

The Deep learning model will be made out of 2 different parts: 

1. A CNN that takes the images as inputs and performs feature extraction.
2. A dense, fully connected layer that will perform classification itself with the features.

In [1]:
#need this to run, don't know why
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

#Check Computer's available devices
#Will need to check in the future
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

#cfg 1
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

#cfg2
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12838017832935611584
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8080208073006711878
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4971491488
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16020402108907825257
physical_device_desc: "device: 0, name: GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2137664995782886590
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
#import some libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import matplotlib.image as mpimg
%matplotlib inline

#image manipulation packages
import cv2
import os
from tqdm import tqdm
from glob import glob
from sklearn.metrics import confusion_matrix,classification_report

#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

#Classification
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix

sns.set()
#import keras
from tensorflow import keras

import glob
import cv2
import shutil
import random
import Augmentor

# Step 2: Label images

In this case, we have a folder with 1132 image - SCIAN-MorphoSpermGS folder - https://cimt.uchile.cl/gold10/. Each image is 35 x 35 pixels and has been classified by 3 experts. We will use majority vote result as target. Each image will need to be loaded and the dataset will need to be created.

#### This will yield a dataset with the picture name and the label.

In [3]:
#Select the path where you've put your dataset provided in Moodle

#step 1: change directory back
os.chdir('../Dataset')
path = os.getcwd()
files = os.listdir(path)

In [4]:
#1 - load txt file
dataframe = pd.read_csv(path + '\PA-expert-annotations.txt', sep = '\\t', header = None, engine = 'python')
dataframe

,0,1,2,3,4
0,p1-pl2-sample01/Sperm_01,5,5,5,5
1,p1-pl2-sample01/Sperm_02,1,5,5,5
2,p1-pl2-sample01/Sperm_03,0,0,0,0
3,p1-pl2-sample01/Sperm_04,1,5,5,5
4,p1-pl2-sample01/Sperm_05,0,0,0,0
...,...,...,...,...,...
1127,p5-pl1-sample20/Sperm_07,1,5,5,5
1128,p5-pl1-sample20/Sperm_09,1,1,5,1
1129,p5-pl1-sample20/Sperm_11,1,1,5,1
1130,p5-pl1-sample20/Sperm_12,5,5,5,5


a. Start by most standard preprocessing of Dataframe

In [5]:
#editions to target dataset
dataframe.rename(columns={0 :"Sperm_Pic", 1: 'Expert_1', 2: 'Expert_2', 3: 'Expert_3', 4: 'Majority_Vote'}, inplace = True)
dataframe['Sperm_Pic'] = dataframe['Sperm_Pic'].str.replace('/S', '-s')

#delete tiny
dataframe = dataframe.loc[dataframe['Majority_Vote'] != 3]
dataframe['Majority_Vote'] = dataframe['Majority_Vote'].replace(5, 3)


#add ch_00 to all rows:
dataframe['Sperm_Pic'] = 'ch00_' + dataframe['Sperm_Pic'] + '.tif'

#Drop all irrelevant features
dataframe = dataframe.drop(['Expert_1', 'Expert_2', 'Expert_3'], axis = 1)

#Show
dataframe

<ipython-input-5-6bf5cf9e776b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Majority_Vote'] = dataframe['Majority_Vote'].replace(5, 3)
<ipython-input-5-6bf5cf9e776b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Sperm_Pic'] = 'ch00_' + dataframe['Sperm_Pic'] + '.tif'


,Sperm_Pic,Majority_Vote
0,ch00_p1-pl2-sample01-sperm_01.tif,3
1,ch00_p1-pl2-sample01-sperm_02.tif,3
2,ch00_p1-pl2-sample01-sperm_03.tif,0
3,ch00_p1-pl2-sample01-sperm_04.tif,3
4,ch00_p1-pl2-sample01-sperm_05.tif,0
...,...,...
1127,ch00_p5-pl1-sample20-sperm_07.tif,3
1128,ch00_p5-pl1-sample20-sperm_09.tif,1
1129,ch00_p5-pl1-sample20-sperm_11.tif,1
1130,ch00_p5-pl1-sample20-sperm_12.tif,3


b. Place images on Folder based on image Label

In [6]:
os.chdir('Partial-Agreement-Images')
path = os.getcwd()

In [7]:
path

'C:\\Users\\Ricardo Santos\\Desktop\\Mestrado Ricardo\\Ano 1\\Spring Semester\\Deep\\Deep_Learning_Project\\Dataset\\Partial-Agreement-Images'

In [8]:
imgs = []

for image in glob.glob(path+'/*.tif'):
        
    imgs.append(cv2.imread(image))
    
    #names of each sample in the image and dataframe are different, gotta deal with this here to match them

    img_name = image.split('\\')[-1]
    check_length = img_name.split('-')[-2]
    if len(check_length) == 7:
        img_name = img_name[:18] + '0' + img_name[18:]
    check_length = img_name.split('-')[-1]
    if len(check_length) == 10:
        img_name = img_name[:-5] + '0' + img_name[-5:]
    img_name = img_name[:-6] + '_' + img_name[-6:]
   
    #path of each class
    path0 = path+'\\class0'
    path1 = path+'\\class1'
    path2 = path+'\\class2'
    path3 = path+'\\class3'
   
    #creates folders to store images from eacg category, if not exists already
    #requires dirs to exist - otherwise enters infinite loop and we have to restrat script
    if not os.path.isdir(path0):

        path = path3
        os.mkdir(path)
        path = path2
        os.mkdir(path)
        path = path1
        os.mkdir(path)
        path = path0
        os.mkdir(path)

    #creates copy of image based on label
    maj_vote = dataframe[dataframe['Sperm_Pic'].str.contains(img_name)]['Majority_Vote']

    if maj_vote.values == 3:
        shutil.copy(image, path3)
    if maj_vote.values == 2:
        shutil.copy(image, path2)
    if maj_vote.values == 1:
        shutil.copy(image, path1)
    if maj_vote.values == 0:
        shutil.copy(image, path0)        

<ipython-input-8-bb3cf003d795>:40: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if maj_vote.values == 3:
<ipython-input-8-bb3cf003d795>:42: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if maj_vote.values == 2:
<ipython-input-8-bb3cf003d795>:44: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if maj_vote.values == 1:
<ipython-input-8-bb3cf003d795>:46: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if maj_vote.values == 0:


In [9]:
#comparing to images on each folder
dataframe['Majority_Vote'].value_counts()

3    656
1    228
0    100
2     76
Name: Majority_Vote, dtype: int64

In [10]:
#Optional Join Hushem Data
from distutils.dir_util import copy_tree

# copy subdirectory example
fromDirectory = r"C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Dataset\HuSHem"
toDirectory = r"C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Dataset\Partial-Agreement-Images"

copy_tree(fromDirectory, toDirectory)

['C:\\Users\\Ricardo Santos\\Desktop\\Mestrado Ricardo\\Ano 1\\Spring Semester\\Deep\\Deep_Learning_Project\\Dataset\\Partial-Agreement-Images\\class0\\image_001.BMP',
 'C:\\Users\\Ricardo Santos\\Desktop\\Mestrado Ricardo\\Ano 1\\Spring Semester\\Deep\\Deep_Learning_Project\\Dataset\\Partial-Agreement-Images\\class0\\image_002.BMP',
 'C:\\Users\\Ricardo Santos\\Desktop\\Mestrado Ricardo\\Ano 1\\Spring Semester\\Deep\\Deep_Learning_Project\\Dataset\\Partial-Agreement-Images\\class0\\image_003.BMP',
 'C:\\Users\\Ricardo Santos\\Desktop\\Mestrado Ricardo\\Ano 1\\Spring Semester\\Deep\\Deep_Learning_Project\\Dataset\\Partial-Agreement-Images\\class0\\image_004.BMP',
 'C:\\Users\\Ricardo Santos\\Desktop\\Mestrado Ricardo\\Ano 1\\Spring Semester\\Deep\\Deep_Learning_Project\\Dataset\\Partial-Agreement-Images\\class0\\image_005.BMP',
 'C:\\Users\\Ricardo Santos\\Desktop\\Mestrado Ricardo\\Ano 1\\Spring Semester\\Deep\\Deep_Learning_Project\\Dataset\\Partial-Agreement-Images\\class0\\image_00

c. Create Folders for Training, Validation and Test Data
The Current solution is somewhat innefficient because it requires the creation of 2 copies of image.

In [27]:
classes_dir = ['/class0', '/class1', '/class2', '/class3']

val_ratio = 0.20
test_ratio = 0.20

if not os.path.isdir(path + '/train'):

    for cls in classes_dir:
    
        #creates train and test folders, with each class separated inside
        os.makedirs(path +'/train' + cls)
        os.makedirs(path +'/val' + cls)
        os.makedirs(path +'/test' + cls)


        # Creating partitions of the data after shuffeling
        src = path + cls # Folder to copy images from

        allFileNames = os.listdir(src)
        np.random.shuffle(allFileNames)
        
        train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - val_ratio - test_ratio)), 
                                                               int(len(allFileNames)* (1 - val_ratio))])
        
        train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
        val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
        test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    
        # Copy-pasting images
        for name in train_FileNames:
            shutil.copy(name, path +'/train' + cls)

        for name in val_FileNames:
            shutil.copy(name, path +'/val' + cls)

        for name in test_FileNames:
            shutil.copy(name, path +'/test' + cls)

In [28]:
#count files in folder

def fileCount(folder):
    "count the number of files in a directory"

    count = 0

    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)

        if os.path.isfile(path):
            count += 1
        elif os.path.isdir(path):
            count += fileCount(path)

    return count

count_0 = fileCount(path+'/train/class0')
count_1 = fileCount(path+'/train/class1')
count_2 = fileCount(path+'/train/class2')
count_3 = fileCount(path+'/train/class3')

print(f'Images Normal: {count_0}\n' +
     f'Images Tapered: {count_1}\n' +
     f'Images Pyriform: {count_2}\n' +
     f'Images Amorphous: {count_3}\n')

Images Normal: 92
Images Tapered: 168
Images Pyriform: 79
Images Amorphous: 424



d. Augment images

In [29]:
# Define augmentation pipelines
class_0 = Augmentor.Pipeline(path+'/train/class0')
class_1 = Augmentor.Pipeline(path+'/train/class1')
class_2 = Augmentor.Pipeline(path+'/train/class2')
class_3 = Augmentor.Pipeline(path+'/train/class3')

# Define different augmentations depending on the pipeline; options are limited since we're working with microscopy data

#visit: https://augmentor.readthedocs.io/en/master/userguide/mainfeatures.html#rotating
#options are:
#Perspective skewing: does not make sense; microscopy data
###Elastic distortion: could work, but I'm afraid it'll actually deform the shape of the sper cell, which is what is picked up to classify it; could be detremental...
###Rotation: makes sense! But no more than 5 degrees left or right...
#Shear: does not make sense?
#Cropping: does not make sense.
###Mirroring: yes! both vertically and horizontally, randomly.


#rotate by a maximum of 5 degrees
class_0.rotate(probability=0.7, max_left_rotation=5, max_right_rotation=5)
class_1.rotate(probability=0.7, max_left_rotation=5, max_right_rotation=5)
class_2.rotate(probability=0.7, max_left_rotation=5, max_right_rotation=5)
class_3.rotate(probability=0.7, max_left_rotation=5, max_right_rotation=5)

#mirroring, vertical or horizontal, randomly
class_0.flip_random(probability=0.7)
class_1.flip_random(probability=0.7)
class_2.flip_random(probability=0.7)
class_3.flip_random(probability=0.7)


# Augment images to the same proportion as existing ones in class 4 (majority class - get to 1000 in each class)
class_0.sample(1024- count_0)
class_1.sample(1024- count_1)
class_2.sample(1024- count_2)
class_3.sample(1024- count_3)

Initialised with 92 image(s) found.
Output directory set to C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Dataset\Partial-Agreement-Images/train/class0\output.Initialised with 168 image(s) found.
Output directory set to C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Dataset\Partial-Agreement-Images/train/class1\output.Initialised with 79 image(s) found.
Output directory set to C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Dataset\Partial-Agreement-Images/train/class2\output.

Processing <PIL.Image.Image image mode=RGB size=131x131 at 0x2DAE54128E0>:   2%|█                                           | 23/932 [00:00<00:09, 99.02 Samples/s]

Initialised with 424 image(s) found.
Output directory set to C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Dataset\Partial-Agreement-Images/train/class3\output.

Processing <PIL.Image.Image image mode=L size=34x34 at 0x2DAE5319190>: 100%|██████████████████████████████████████████████| 932/932 [00:01<00:00, 542.18 Samples/s]
Processing <PIL.Image.Image image mode=L size=35x35 at 0x2DAE7B1E040>: 100%|██████████████████████████████████████████████| 856/856 [00:01<00:00, 542.11 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=131x131 at 0x2DAE5696490>: 100%|██████████████████████████████████████████| 945/945 [00:01<00:00, 569.28 Samples/s]
Processing <PIL.Image.Image image mode=L size=35x35 at 0x2DAE53190A0>: 100%|██████████████████████████████████████████████| 600/600 [00:01<00:00, 518.58 Samples/s]


## Step 3 - Load Data

In [30]:
#load image
from keras.preprocessing.image import ImageDataGenerator

#to play around with these
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 5, #rotates images from -5 to 5 degrees
                                   width_shift_range = 0.06, #translates images by 6% to left or right
                                   height_shift_range = 0.06, #translates images by 6% up and down
                                   vertical_flip = True,
                                   horizontal_flip = True,
                                   brightness_range=[0.2,1.2], 
                                   fill_mode='nearest',
                                   zoom_range = 0.2,
                                   ) 

val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

#test different color maps -  class modes and cross validation types
training_set = train_datagen.flow_from_directory(path+'/train',
                                                 target_size = (32, 32),
                                                 batch_size = 32,
                                                 shuffle = True,
                                                 class_mode="categorical",
                                                 color_mode = 'rgb')

val_set = val_datagen.flow_from_directory(path+'/val',
                                            target_size = (32, 32),
                                            batch_size = 32,
                                            shuffle = True,
                                            class_mode="categorical",
                                            color_mode = 'rgb')

test_set = test_datagen.flow_from_directory(path+'/test',
                                            target_size = (32, 32),
                                            batch_size = 1,
                                            shuffle = True,
                                            class_mode="categorical",
                                            color_mode = 'rgb')

Found 4096 images belonging to 4 classes.
Found 256 images belonging to 4 classes.
Found 257 images belonging to 4 classes.


## Step 4 - Very Simple Test Model

To Test CROSS_VAL

In [35]:
def cnn_test_model():
    '''creates an image classification model that uses 2 Convolutional CNNs layers (with maxpooling) and feeds the data through
    a dense connected layer. This is a simple model to compute fast to test cross_validation'''
    
    cnn_model = keras.Sequential([

        #convolutional layer with 32 3x3 filters - again, arbitrary,
        keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding = 'same', activation='relu'), 
        keras.layers.BatchNormalization(),
        
        #2nd convolution with 128 filters
        keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding = 'same', activation= 'relu'),
        keras.layers.BatchNormalization(),
       
        #MaxPooling - takes the max value of each 2x2 pool in the feature map
        keras.layers.MaxPool2D(pool_size=(2,2)),
        keras.layers.BatchNormalization(),
        
        #the result of kthe CNN is then flattened and placed into the 
        keras.layers.Flatten(),
        
        keras.layers.Dense(256, activation='relu'),
        keras.layers.BatchNormalization(),
        
        #final layer, is output, 1 out of 4 possible results
        #0 Normal, 1 Tapered, 2 Pyriform, 3 Amorphous
        keras.layers.Dense(4, activation='softmax')
    ])
    
    return cnn_model

In [36]:
# define call-back early stopping criteria
es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=300, restore_best_weights = True)

In [37]:
#compile model: First try
test_model = cnn_test_model()

test_model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#fit model
history = test_model.fit(training_set,
        epochs=4000,
        validation_data=val_set,
        verbose = 1, 
        validation_steps = len(val_set),
        callbacks = [es])

# evaluate the model
_, train_acc = test_model.evaluate(training_set, verbose=1)
_, val_acc = test_model.evaluate(val_set, verbose=1)
print('Train: %.3f, Val: %.3f' % (train_acc, val_acc))

# plot training history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

#saves model
test_model.save(r"C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Models\test_model.h5")

Epoch 1/4000
128/128 [==============================] - 4s 33ms/step - loss: 0.7070 - accuracy: 0.7131 - val_loss: 20.8013 - val_accuracy: 0.5156
Epoch 2/4000
128/128 [==============================] - 4s 32ms/step - loss: 0.6860 - accuracy: 0.7302 - val_loss: 1.0778 - val_accuracy: 0.5469
Epoch 3/4000
128/128 [==============================] - 5s 37ms/step - loss: 0.6210 - accuracy: 0.7549 - val_loss: 1.0433 - val_accuracy: 0.6016
Epoch 4/4000
128/128 [==============================] - 4s 34ms/step - loss: 0.6113 - accuracy: 0.7549 - val_loss: 0.9794 - val_accuracy: 0.6094
Epoch 5/4000
128/128 [==============================] - 4s 33ms/step - loss: 0.5998 - accuracy: 0.7690 - val_loss: 0.8928 - val_accuracy: 0.6367
Epoch 6/4000
128/128 [==============================] - 4s 32ms/step - loss: 0.5729 - accuracy: 0.7686 - val_loss: 1.0363 - val_accuracy: 0.6328
Epoch 7/4000
128/128 [==============================] - 4s 32ms/step - loss: 0.5518 - accuracy: 0.7820 - val_loss: 0.9534 - val_a

128/128 [==============================] - 4s 34ms/step - loss: 0.4238 - accuracy: 0.8369 - val_loss: 1.5092 - val_accuracy: 0.5312
Epoch 57/4000
128/128 [==============================] - 4s 34ms/step - loss: 0.5344 - accuracy: 0.7983 - val_loss: 1.1769 - val_accuracy: 0.6094
Epoch 58/4000
128/128 [==============================] - 4s 33ms/step - loss: 0.4145 - accuracy: 0.8447 - val_loss: 1.0186 - val_accuracy: 0.6602
Epoch 59/4000
128/128 [==============================] - 4s 33ms/step - loss: 0.4102 - accuracy: 0.8423 - val_loss: 0.9967 - val_accuracy: 0.6602
Epoch 60/4000
128/128 [==============================] - 4s 34ms/step - loss: 0.4107 - accuracy: 0.8428 - val_loss: 1.0165 - val_accuracy: 0.6289
Epoch 61/4000
128/128 [==============================] - 4s 34ms/step - loss: 0.3856 - accuracy: 0.8491 - val_loss: 1.1403 - val_accuracy: 0.5859
Epoch 62/4000
128/128 [==============================] - 4s 34ms/step - loss: 0.3966 - accuracy: 0.8459 - val_loss: 1.1481 - val_accuracy:

In [ ]:
#Performance do primeiro teste

test_loss, test_acc = test_model.evaluate(test_set)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
#Model Summary
test_model.summary()

In [ ]:
#get predicted probality
prediction = test_model.predict(test_set,verbose=1)

#Get class of prediction
predicted_class = np.argmax(prediction,axis=1)

# get names of pictures
filenames = test_set.filenames

#get trueclass
true_classes = test_set.classes

#store info in dataframe
df_predictions = pd.DataFrame({'Filename': filenames, 'Label': true_classes, 'Test': predicted_class})

In [ ]:
df_predictions

In [ ]:
#confusion matrix and classification report
class_labels = list(test_set.class_indices.keys())   
print(confusion_matrix(true_classes, predicted_class))

report = classification_report(true_classes, predicted_class, target_names=class_labels)
print(report)

## Step 5 - Design a More Complex and Robust Models

In [ ]:
def image_cnn_standard_model():
    '''creates an image classification model that uses 2 Convolutional CNNs layers (with maxpooling) and feeds the data through
    a dense connected layer. This is trial and error there is no specific reason for 2 layers'''
    cnn_model = keras.Sequential([

        #convolutional layer with 32 3x3 filters - again, arbitrary,
        keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding = 'same', activation='relu'), 
        keras.layers.BatchNormalization(),
        
        #MaxPooling - takes the max value of each 2x2 pool in the feature map
        keras.layers.MaxPool2D(pool_size=(2,2)),
        keras.layers.BatchNormalization(),
        
        #3rd convolution with 64 filters
        keras.layers.Conv2D(filters=64, kernel_size=(3,3),  padding = 'same', activation= 'relu'),
        keras.layers.BatchNormalization(),
        
        #4rd convolution with 128 filters
        keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding = 'same', activation= 'relu'),
        keras.layers.BatchNormalization(),
        
        #Second MaxPool2D - to check with other options
        keras.layers.MaxPool2D(pool_size=(2,2)),
        keras.layers.BatchNormalization(),
        
        #5th convolutional layer with 24 3x3 filters - again, arbitrary,
        keras.layers.Conv2D(filters=32, kernel_size=(4,4), padding = 'same', activation='relu'), 
        keras.layers.BatchNormalization(),
        
        #MaxPooling - takes the max value of each 2x2 pool in the feature map
        #keras.layers.MaxPool2D(pool_size=(2,2)),

        #second convolution with 36 filters
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding = 'same', activation= 'relu'),
        keras.layers.BatchNormalization(),
        
        keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding = 'same', activation= 'relu'),
        keras.layers.BatchNormalization(),
        
        #Second MaxPool2D - to check with other options
        keras.layers.MaxPool2D(pool_size=(2,2)),
        keras.layers.BatchNormalization(),
        
        #convolutional layer with 24 3x3 filters - again, arbitrary,
        keras.layers.Conv2D(filters=128, kernel_size=(4,4), padding = 'same', activation='relu'), 
        keras.layers.BatchNormalization(),
        

        #second convolution with 36 filters
        keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding = 'same', activation= 'relu'),
        keras.layers.BatchNormalization(),
        
        keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding = 'same', activation= 'relu'),
        keras.layers.BatchNormalization(),
        
        #Second MaxPool2D - to check with other options
        keras.layers.MaxPool2D(pool_size=(2,2)),
        keras.layers.BatchNormalization(),
        
        #the result of kthe CNN is then flattened and placed into the 
        keras.layers.Flatten(),
        
        keras.layers.Dense(128, activation='relu'),
        keras.layers.BatchNormalization(),
        #Add Dropout
        keras.layers.Dropout(0.4),
        
        keras.layers.Dense(256, activation='relu'),
        keras.layers.BatchNormalization(),
        #final layer, is output, 1 out of 5 possible results
        #0 Normal, 1 Tapered, 2 Pyriform, 3 Amorphous
        keras.layers.Dense(4, activation='softmax')
    ])
    
    return cnn_model

In [ ]:
#compile model: First try
cnn_model = image_cnn_standard_model()

cnn_model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#fit model
history = cnn_model.fit(training_set,
        epochs=4000,
        validation_data=val_set,
        verbose = 1, 
        validation_steps = len(val_set),
        callbacks = [es])

# evaluate the model
_, train_acc = cnn_model.evaluate(training_set, verbose=1)
_, val_acc = cnn_model.evaluate(val_set, verbose=1)
print('Train: %.3f, Val: %.3f' % (train_acc, val_acc))

# plot training history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()


cnn_model.save(r"C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Models\model_1.h5")

In [ ]:
#Model Summary
cnn_model.summary()

In [ ]:
#test performance

test_loss, test_acc = cnn_model.evaluate(test_set)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

### Make Predictions

Code still needs improvement

These predictions are gated by batch size

Let's try and improve it with bruteforce Hyperparam optimization whiole being on the lookout for other ways to improve performance. Model is predicting all as Amorphous.

In [ ]:
#get predicted probality
prediction = cnn_model.predict(test_set,verbose=1)

#Get class of prediction
predicted_class = np.argmax(prediction,axis=1)

#get trueclass
true_classes = test_set.classes

# get names of pictures
filenames = test_set.filenames

#store info in dataframe
robust_predictions = pd.DataFrame({'Filename': filenames,'Model_1': predicted_class})

In [ ]:
#join prediction to dataframe
df_predictions = df_predictions.merge(robust_predictions, on = 'Filename', how = 'inner')

del robust_predictions

In [ ]:
df_predictions 

In [ ]:
#confusion matrix and classification report
class_labels = list(test_set.class_indices.keys())   
print(confusion_matrix(true_classes, predicted_class))

report = classification_report(true_classes, predicted_class, target_names=class_labels)
print(report)

# Step 6 - Benchmarks and other tests

Try and compare the models performance with other standardly used models:

AlexNet

In [ ]:
#ALEX NET

#Importing library 
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization


#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(4))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

In [ ]:
# Compiling the model
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [ ]:
#fit model

history = AlexNet.fit(training_set,
        epochs=4000,
        validation_data=val_set,
        verbose = 1, 
        validation_steps = len(val_set),
        callbacks = [es])

# evaluate the model
_, train_acc = AlexNet.evaluate(training_set, verbose=1)
_, val_acc = AlexNet.evaluate(val_set, verbose=1)
print('Train: %.3f, Val: %.3f' % (train_acc, val_acc))

# plot training history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

AlexNet.save(r"C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Models\Alex.h5")

In [ ]:
test_loss, test_acc = AlexNet.evaluate(test_set)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
#get predicted probality
prediction = AlexNet.predict(test_set,verbose=1)

#Get class of prediction
predicted_class = np.argmax(prediction,axis=1)

#get trueclass
true_classes = test_set.classes

# get names of pictures
filenames = test_set.filenames

#store info in dataframe
alex_predictions = pd.DataFrame({'Filename': filenames,'AlexNet': predicted_class})

In [ ]:
#join prediction to dataframe
df_predictions = df_predictions.merge(alex_predictions, on = 'Filename', how = 'inner')

del alex_predictions

In [ ]:
df_predictions

In [ ]:
#confusion matrix and classification report
class_labels = list(test_set.class_indices.keys())   
print(confusion_matrix(true_classes, predicted_class))

report = classification_report(true_classes, predicted_class, target_names=class_labels)
print(report)

## Transfer Learning with RESNET50

In [ ]:
#resnet50 requires its own preprocessing 
from keras.applications.resnet50 import preprocess_input

#to play around with these
res50train_datagen = ImageDataGenerator(rescale = 1./255,
                                   preprocessing_function=preprocess_input,
                                   rotation_range = 5,
                                   width_shift_range = 0.06, 
                                   height_shift_range = 0.06, 
                                   vertical_flip = True,
                                   horizontal_flip = True,
                                   brightness_range=[0.2,1.2], 
                                   fill_mode='nearest',
                                   zoom_range = 0.2,
                                   ) 

res50val_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)
res50test_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

#test different color maps -  class modes and cross validation types
res_train = res50train_datagen.flow_from_directory(path+'/train',
                                                 target_size = (32, 32),
                                                 batch_size = 32,
                                                 class_mode="categorical",
                                                 color_mode = 'rgb',
                                                 shuffle = True)

res_val = res50val_datagen.flow_from_directory(path+'/val',
                                            target_size = (32, 32),
                                            batch_size = 32,
                                            class_mode="categorical",
                                            color_mode = 'rgb',
                                            shuffle = True)

res_test = res50test_datagen.flow_from_directory(path+'/test',
                                            target_size = (32, 32),
                                            batch_size = 1,
                                            class_mode="categorical",
                                            color_mode = 'rgb',
                                            shuffle = True)

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50


res50model = Sequential()

#Layer 1: RES50 without top layer
res50model.add(ResNet50(weights='imagenet', input_shape= (32,32,3),
                 include_top = False, classes=5,))

#Layer 2: RES50 without top layer
res50model.add(Flatten())
res50model.add(Dense(4, activation = 'softmax'))

#freeze layers in resnet - weights obtained with IMAGENET challenge, we only train final layer
res50model.layers[0].trainable = False

In [ ]:
res50model.summary()

In [ ]:
res50model.output_shape

In [ ]:
#compile

res50model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [ ]:
#fit model

res_history = res50model.fit(res_train,
        epochs=4000,
        validation_data=res_val,
        verbose = 1, 
        validation_steps = len(val_set),
        callbacks = [es])

# evaluate the model
_, train_acc = res50model.evaluate(training_set, verbose=1)
_, val_acc = res50model.evaluate(val_set, verbose=1)
print('Train: %.3f, Val: %.3f' % (train_acc, val_acc))

# plot training history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

# Calling `save('res50_net.h5')`.
res50model.save(r"C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Models\Res50_net.h5")

In [ ]:
test_loss, test_acc = res50model.evaluate(res_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
#get predicted probality
prediction = res50model.predict(res_test,verbose=1)

#Get class of prediction
predicted_class = np.argmax(prediction,axis=1)

#get trueclass
true_classes = res_test.classes

# get names of pictures
filenames = res_test.filenames

#store info in dataframe
res50_prebuilt_predictions = pd.DataFrame({'Filename': filenames,'ResNet50': predicted_class})

In [ ]:
#join prediction to dataframe
df_predictions = df_predictions.merge(res50_prebuilt_predictions, on = 'Filename', how = 'inner')

del res50_prebuilt_predictions

In [ ]:
df_predictions

In [ ]:
#confusion matrix and classification report
class_labels = list(test_set.class_indices.keys())   
print(confusion_matrix(true_classes, predicted_class))

report = classification_report(true_classes, predicted_class, target_names=class_labels)
print(report)

## RESNET50 Trained from Scratch

In [ ]:
res50model_2 = Sequential()

#Layer 1: RES50 without top layer
res50model_2.add(ResNet50(weights=None, input_shape= (32,32,3),
                 include_top = False, classes=5,))

#Layer 2: RES50 without top layer
res50model_2.add(Flatten())
res50model_2.add(Dense(4, activation = 'softmax'))

In [ ]:
res50model_2.summary()

In [ ]:
#compile

res50model_2.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [ ]:
#fit model

res_history_2 = res50model_2.fit(res_train,
        epochs=4000,
        validation_data=res_val,
        verbose = 1, 
        validation_steps = len(res_val),
        callbacks = [es])

# evaluate the model
_, train_acc = res50model_2.evaluate(res_train, verbose=1)
_, val_acc = res50model_2.evaluate(res_val, verbose=1)
print('Train: %.3f, Val: %.3f' % (train_acc, val_acc))

# plot training history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
res50model.save(r"C:\Users\Ricardo Santos\Desktop\Mestrado Ricardo\Ano 1\Spring Semester\Deep\Deep_Learning_Project\Models\Res50_from_scratch.h5")

In [ ]:
#Performance do primeiro teste

test_loss, test_acc = res50model_2.evaluate(res_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
#get predicted probality
prediction = res50model_2.predict(res_test,verbose=1)

#Get class of prediction
predicted_class = np.argmax(prediction,axis=1)

#get trueclass
true_classes = res_test.classes

# get names of pictures
filenames = res_test.filenames

#store info in dataframe
res50_predictions = pd.DataFrame({'Filename': filenames,'ResNet50 From Scratch': predicted_class})

In [ ]:
#join prediction to dataframe
df_predictions = df_predictions.merge(res50_predictions, on = 'Filename', how = 'inner')

del res50_predictions

In [ ]:
df_predictions

In [ ]:
#confusion matrix and classification report
class_labels = list(test_set.class_indices.keys())   
print(confusion_matrix(true_classes, predicted_class))

report = classification_report(true_classes, predicted_class, target_names=class_labels)
print(report)

### FOR TOMORROW

Check different loss functions
CROSS VALIDATION

